最近对赚钱的话题很感兴趣，在知乎上关注了很多“赚钱”相关的问题，高质量的有不少，但是夹杂着私货的也不少。不过知乎的数据比较全，我们完全可以用来做文本分析。

爬虫的原理我就不细讲了，如果大家对爬虫的原理和相关库不甚了解，又想快速入门，不妨看看我们这门课。

### 待爬网址
https://www.zhihu.com/question/34011097

![](img/首图.png)

## 分析请求
因为我们知道知乎的响应数据都是json型网站，所以我们想找到json对应的链接规律。F12键打开开发者工具，选中XHR,不停地往下滑动页面，开发者工具Network会不停的闪过很多链接。
![](img/快速往下滑.gif)
经过排查我们发现这个链接很特殊，点击进去详情如下
![](img/url.png)
对应的数据是json格式
![](img/datalist.png)
里面果然是用户的回答数据
![](img/data.png)

现在我们将找到的网址复制下来分析分析

```
https://www.zhihu.com/api/v4/questions/34011097/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&limit=5&offset=10&platform=desktop&sort_by=default
```

我们发现网址中有  **34011097** 和 **offset** 两个参数是可以调整的，分别代表**话题id**和 **回答的id**。我们将上面的网址整理一下，设计成网址模板base_url

```python
base_url = 'https://www.zhihu.com/api/v4/questions/{question_id}/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&limit=5&offset={offset}&platform=desktop&sort_by=default'
```

## 伪装请求
我们还要注意的一点是写爬虫一般是需要伪装请求头headers，而在知乎这种网站，我们可能还需要cookies。我新建了一个 **settings.py**文件，用来存放cookies、headers、网址模板base_url和question_id


```
COOKIES = {'cookie': '你的cookies'}

HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}

BASE_URL = 'https://www.zhihu.com/api/v4/questions/{question_id}/answers?include=data%5B%2A%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%3Bdata%5B%2A%5D.mark_infos%5B%2A%5D.url%3Bdata%5B%2A%5D.author.follower_count%2Cbadge%5B%2A%5D.topics&limit=5&offset={offset}&platform=desktop&sort_by=default'

QUESTION_ID = '34011097'
```

## 数据存储
接下来我们新建 **zhihu.py**用于设计爬虫逻辑，因为知乎的数据都是json格式，相对于html而言json的数据更有层次性更加的干净。为了保证后续数据分析的可扩展性，我们尽量保存原始。所以这里用到了jsonlines库用于存储json数据(以行的方式存储每个json)，如果不熟悉可以把 [jsonlines库：高效率的保存多个python对象](http://mp.weixin.qq.com/s?__biz=MzI1MTE2ODg4MA==&mid=2650070130&idx=1&sn=ba217455f8cf209fe6ed77bfc1e04578&chksm=f1f76d6dc680e47bb6e7d6095a9f7fee593d9d640d5f1670b5df10d1c8b0dfa3ce3d4354d87c&scene=21#wechat_redirect) 这篇文章收藏起来。



## 知乎爬虫代码
在 **zhihu.py** 文件中

In [ ]:
import requests
import json
#从settings.py文件中读取用户自定义的参数
#settings.py一般需要设置的参数包括cookies和question_id
from settings import BASE_URL, HEADERS, COOKIES, QUESTION_ID
import jsonlines


class Zhihu(object):
    def __init__(self, offset):
        #offset相当于第几页，比如从第一页开始就设置为0.
        self.offset = offset

    def crawl(self):
        #采集函数
        url = BASE_URL.format(question_id=QUESTION_ID, offset=self.offset)
        req = requests.get(url, headers=HEADERS, cookies=COOKIES)
        jsondata = req.json()
        if not jsondata['paging']['is_end']:
            self.save(jsondata['data'])
            self.offset += 1
            return self.crawl()
        else:
            print('程序运行终止', )
            return


    def save(self, jsondata):
        #存储函数,存储到  QUESTION_ID.json 文件当中
        with jsonlines.open('{}.json'.format(QUESTION_ID), 'a') as writer:
            for jsonl in jsondata:
                print(self.offset, jsonl)
                writer.write(jsonl)


#从第一页开始抓取
Zhihu(offset=0).crawl()

## 程序运行
![](img/抓取.gif)

## 读取抓取json数据
这里使用jsonlines库，该库以行的方式读取，得到的是可迭代对象。如果不熟悉可以把 [jsonlines库：高效率的保存多个python对象](http://mp.weixin.qq.com/s?__biz=MzI1MTE2ODg4MA==&mid=2650070130&idx=1&sn=ba217455f8cf209fe6ed77bfc1e04578&chksm=f1f76d6dc680e47bb6e7d6095a9f7fee593d9d640d5f1670b5df10d1c8b0dfa3ce3d4354d87c&scene=21#wechat_redirect) 这篇文章收藏起来。将其转化为dataframe类型，这样我们就可以用pandas库进行读取

In [15]:
import pandas as pd
import jsonlines 

#这次我们抓的话题id是34011097，所以存储数据的文件名为  34011097.json
with jsonlines.open('34011097.json', 'r') as jsonf:
    df = pd.DataFrame(jsonf)
    
#只看前5行
df.head()

,admin_closed_comment,annotation_action,answer_type,author,can_comment,collapse_reason,collapsed_by,comment_count,comment_permission,content,...,question,relationship,relevant_info,reshipment_settings,reward_info,suggest_edit,type,updated_time,url,voteup_count
0,False,[],normal,"{'id': '1d021e70bba109aca30cae3d1d2fa32d', 'ur...","{'reason': '', 'status': True}",,nobody,1160,all,<p><b>统一回复了一些知友的问题，请拉到文章底部</b></p><p><b>大学赚1W真...,...,"{'type': 'question', 'id': 34011097, 'title': ...","{'is_author': False, 'is_authorized': False, '...","{'is_relevant': False, 'relevant_type': '', 'r...",allowed,"{'can_open_reward': False, 'is_rewardable': Tr...","{'reason': '', 'status': False, 'tip': '', 'ti...",answer,1536860750,https://www.zhihu.com/api/v4/answers/399375806,17515
1,False,[],normal,"{'id': '3903a1117d2503e0d96a7baa5bc87353', 'ur...","{'reason': '', 'status': True}",,nobody,881,all,<p></p><p>岁末年终，以此回顾大学的艰苦岁月，也为余下的半年不到时光开个新头。<br...,...,"{'type': 'question', 'id': 34011097, 'title': ...","{'is_author': False, 'is_authorized': False, '...","{'is_relevant': False, 'relevant_type': '', 'r...",need_payment,"{'can_open_reward': False, 'is_rewardable': Tr...","{'reason': '', 'status': False, 'tip': '', 'ti...",answer,1545532105,https://www.zhihu.com/api/v4/answers/137453313,8523
2,False,[],normal,"{'id': '1ef9fc4a95afe29e156a71b0fb9f06ed', 'ur...","{'reason': '', 'status': True}",,nobody,2636,all,<b>补充：</b><b>统一回答知友评论和私信的问题，请拉到本文底部。</b><b>本文多...,...,"{'type': 'question', 'id': 34011097, 'title': ...","{'is_author': False, 'is_authorized': False, '...","{'is_relevant': False, 'relevant_type': '', 'r...",disallowed,"{'can_open_reward': False, 'is_rewardable': Fa...","{'reason': '', 'status': False, 'tip': '', 'ti...",answer,1442358895,https://www.zhihu.com/api/v4/answers/62277897,31693
3,False,[],normal,"{'id': 'b4fbe8321dfbcf82c6138aa7c7c1fedf', 'ur...","{'reason': '', 'status': True}",,nobody,99,all,<p><b>史上最全兼职汇总来袭！！</b></p><p><b>深入分析17种兼职的优缺点网...,...,"{'type': 'question', 'id': 34011097, 'title': ...","{'is_author': False, 'is_authorized': False, '...","{'is_relevant': False, 'relevant_type': '', 'r...",need_payment,"{'can_open_reward': False, 'is_rewardable': Fa...","{'reason': '', 'status': False, 'tip': '', 'ti...",answer,1500732828,https://www.zhihu.com/api/v4/answers/145503363,3202
4,False,[],normal,"{'id': '492f24eb709538a7f09e579ffe8e103d', 'ur...","{'reason': '', 'status': True}",,nobody,676,all,<p>为什么只赚一万块？！</p><p>只赚一万块，学费生活费是有了，那谈恋爱的花销你考虑没...,...,"{'type': 'question', 'id': 34011097, 'title': ...","{'is_author': False, 'is_authorized': False, '...","{'is_relevant': False, 'relevant_type': '', 'r...",disallowed,"{'can_open_reward': False, 'is_rewardable': Fa...","{'reason': '', 'status': False, 'tip': '', 'ti...",answer,1507773190,https://www.zhihu.com/api/v4/answers/243129056,41222


In [18]:
df.drop_duplicates(subset=['author'], keep='first', inplace=True)

TypeError: unhashable type: 'dict'

In [17]:
df.describe()

,comment_count,created_time,id,updated_time,voteup_count
count,36.000000,3.600000e+01,3.600000e+01,3.600000e+01,36.000000
mean,441.555556,1.492528e+09,2.199633e+08,1.507912e+09,5178.194444
std,595.222573,3.025130e+07,1.546337e+08,2.952898e+07,8969.575906
min,23.000000,1.440012e+09,5.999026e+07,1.441620e+09,111.000000
25%,59.250000,1.467479e+09,1.090351e+08,1.494296e+09,381.000000
50%,243.500000,1.488049e+09,1.484671e+08,1.511436e+09,1745.000000
75%,511.750000,1.517499e+09,3.091221e+08,1.527414e+09,4075.750000
max,2636.000000,1.551151e+09,6.085812e+08,1.551236e+09,41222.000000


![](img/dataframe.gif)